In [1]:
import niimpy
df = niimpy.reading.read.read_mhealth_total_sleep_time_from_file("mhealth_test.json")
df

,total_sleep_time,total_sleep_time_unit,effective_start_time,effective_end_time,descriptive_statistic,descriptive_statistic_denominator,effective_time_frame.duration.value,effective_time_frame.duration.unit,effective_time_frame.end_date_time,start_time,duration
timestamp,,,,,,,,,,,
2016-02-05 20:35:00-08:00,465.00,min,2016-02-05 20:35:00-08:00,2016-02-06 06:35:00-08:00,NaN,NaN,NaN,NaN,NaN,2016-02-05 20:35:00-08:00,0 days 10:00:00
2016-02-05 07:00:00-08:00,7.25,h,2016-02-05 07:00:00-08:00,2016-06-06 07:00:00-08:00,average,d,NaN,NaN,NaN,2016-02-05 07:00:00-08:00,122 days 00:00:00
NaT,3.00,h,NaT,NaT,NaN,NaN,10.0,d,2013-02-05T07:35:00Z,None,None


In [2]:
import json
import niimpy
import pandas as pd
with open("mhealth_test.json") as f:
        data = json.load(f)
data.append({
        "total_sleep_time": {"value":3, "unit": "h"},"effective_time_frame": {
            "duration": {
                "value": 10,
                "unit": "d"
            },
            "end_date_time": "2013-02-05T07:35:00Z"
        }
})

df = pd.json_normalize(data)
prefix = "effective_time_frame.time_interval"
start_col = f'{prefix}.start_date_time'
end_col = f'{prefix}.end_date_time'

rows = ~df[start_col].isnull()
df.loc[rows, "start_time"] = pd.to_datetime(df.loc[rows, start_col])

rows = ~df[end_col].isnull()
df.loc[rows, end_col] = pd.to_datetime(df.loc[rows, end_col])

rows = ~df[end_col].isnull() & ~df[start_col].isnull()
df.loc[rows, "duration"] = pd.to_datetime(df.loc[rows, end_col]) - pd.to_datetime(df.loc[rows, "start_time"])

df["duration"]

0     0 days 10:00:00
1   122 days 00:00:00
2                 NaT
3                 NaT
Name: duration, dtype: timedelta64[ns]